In [1]:
import pandas as pd 
import folium
import os
import geohash
from folium.plugins import HeatMap

In [2]:
SAFECAST_DATA_PATH = os.path.join('results', 'first_million_with_elevation.csv')
df = pd.read_csv(SAFECAST_DATA_PATH)

In [3]:
df.head()

,captured time,latitude,longitude,value,elevation
0,2018-10-21 01:00:26.000000,36.041080,140.226816,23.0,31.0
1,2018-10-21 01:00:22.000000,37.796306,140.514413,19.0,72.0
2,2018-10-21 01:00:16.000000,37.723330,140.476797,15.0,141.0
3,2018-10-21 00:59:16.000000,52.444900,13.315000,16.0,47.0
4,2018-10-21 01:00:16.000000,37.787500,140.552400,18.0,107.0


In [4]:
df.shape

(806849, 5)

In [5]:
def compute_geohash(row):
    return geohash.encode(row["latitude"], row["longitude"], precision=4)

In [6]:
df["geohash"] = df.apply(compute_geohash, axis=1)

In [7]:
df.head()

,captured time,latitude,longitude,value,elevation,geohash
0,2018-10-21 01:00:26.000000,36.041080,140.226816,23.0,31.0,xn7t
1,2018-10-21 01:00:22.000000,37.796306,140.514413,19.0,72.0,xnez
2,2018-10-21 01:00:16.000000,37.723330,140.476797,15.0,141.0,xney
3,2018-10-21 00:59:16.000000,52.444900,13.315000,16.0,47.0,u336
4,2018-10-21 01:00:16.000000,37.787500,140.552400,18.0,107.0,xney


In [8]:
grouped_cor = df[["geohash", "value", "elevation"]].groupby("geohash").corr().iloc[0::2,-1].to_frame().reset_index()[['geohash', 'value']]
grouped_cor.head()

,geohash,value
0,8e8y,NaN
1,9exz,NaN
2,9mum,0.144442
3,9muq,-0.011311
4,9ppb,NaN


In [9]:
grouped_cor_filtered = grouped_cor[grouped_cor["value"].notna()]

In [10]:
grouped_cor_filtered.head()

,geohash,value
2,9mum,0.144442
3,9muq,-0.011311
5,9q4g,-0.026391
6,9q55,0.005160
7,9q5c,-0.662545


In [11]:
def compute_geohash_middle_lat(row):
    box = geohash.bbox(row.geohash)
    return (box['s'] + box['n']) / 2
def compute_geohash_middle_lon(row):
    box = geohash.bbox(row.geohash)
    return (box['w'] + box['e']) / 2

In [12]:
grouped_cor_filtered["lat"] = grouped_cor_filtered.apply(compute_geohash_middle_lat, axis=1)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [13]:
grouped_cor_filtered.head()

,geohash,value,lat
2,9mum,0.144442,33.310547
3,9muq,-0.011311,33.486328
5,9q4g,-0.026391,34.365234
6,9q55,0.005160,34.365234
7,9q5c,-0.662545,34.013672


In [14]:
grouped_cor_filtered["lon"] = grouped_cor_filtered.apply(compute_geohash_middle_lon, axis=1)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [15]:
grouped_cor_filtered.head()

,geohash,value,lat,lon
2,9mum,0.144442,33.310547,-117.597656
3,9muq,-0.011311,33.486328,-117.597656
5,9q4g,-0.026391,34.365234,-119.707031
6,9q55,0.005160,34.365234,-119.355469
7,9q5c,-0.662545,34.013672,-118.300781


In [17]:
grouped_cor_filtered['value'] += 1
grouped_cor_filtered.head()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,geohash,value,lat,lon
2,9mum,1.144442,33.310547,-117.597656
3,9muq,0.988689,33.486328,-117.597656
5,9q4g,0.973609,34.365234,-119.707031
6,9q55,1.005160,34.365234,-119.355469
7,9q5c,0.337455,34.013672,-118.300781


In [18]:
max_amount = float(grouped_cor_filtered['value'].max())

hmap = folium.Map(location=[37.7503, 140.4676], zoom_start=7, )

hm_wide = HeatMap( list(zip(grouped_cor_filtered.lat.values, grouped_cor_filtered.lon.values, grouped_cor_filtered.value.values)),
                   min_opacity=0.2,
                   max_val=max_amount,
                   radius=17, blur=15, 
                   max_zoom=1, 
                 )

hmap.add_child(hm_wide)